<a href="https://colab.research.google.com/github/avadacodavra/MalariaDetectionBioproj/blob/main/Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score

# Function to extract features using VGG19
def extract_vgg19_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = vgg19.predict(img_array)
    return features.flatten()

# Load pre-trained VGG19 model
vgg19 = VGG19(weights='imagenet', include_top=False, pooling='avg')

# Edit this path to your image dataset directory
dataset_path = '/content/sample_data'

# Initialize lists to store features and labels
features = []
labels = []

# Iterate through each image in the dataset
for folder in os.listdir(dataset_path):
    if os.path.isdir(os.path.join(dataset_path, folder)):
        label = folder
        folder_path = os.path.join(dataset_path, folder)
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            img_features = extract_vgg19_features(img_path)
            features.append(img_features)
            labels.append(label)

# Convert lists to arrays
X = np.array(features)
y = np.array(labels)

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

# Make predictions
y_pred = svm_classifier.predict(X_test)

# Calculate precision and recall
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

1/1 [==============================] - 1s 574ms/step
Precision: 0.9714795008912657
Recall: 0.9696969696969697


In [ ]:
# Function to do prediction on a new image
def predict_image(img_path, scaler, svm_classifier):
    # Extract features
    img_features = extract_vgg19_features(img_path)
    # Normalize features
    normalized_features = scaler.transform([img_features])
    # Make prediction
    prediction = svm_classifier.predict(normalized_features)
    return prediction[0]

# Example usage
new_image_path = '/content/sample_data/C189P150ThinF_IMG_20151203_142224_cell_92.png'
prediction = predict_image(new_image_path, scaler, svm_classifier)
print("Predicted class:", prediction)

1/1 [==============================] - 1s 1s/step
Predicted class: parasitic
